#0. Data preparation

In [1]:
import os
import subprocess
import torchvision.transforms as transforms
from PIL import Image
from torchvision.datasets import CIFAR10

In [2]:
dataset_path = './datasets/cifar10'
train_path = os.path.join(dataset_path, 'train')
val_path = os.path.join(dataset_path, 'val')

In [3]:
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

In [4]:
transform = transforms.Compose([transforms.ToTensor()])

In [5]:
trainset = CIFAR10(root=train_path, train=True, download=True, transform=transform)
valset = CIFAR10(root=val_path, train=False, download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:12<00:00, 13492767.54it/s]


Extracting ./datasets/cifar10/train/cifar-10-python.tar.gz to ./datasets/cifar10/train


100%|██████████| 170498071/170498071 [00:12<00:00, 13352493.10it/s]


Extracting ./datasets/cifar10/val/cifar-10-python.tar.gz to ./datasets/cifar10/val


In [6]:
def save_images(dataset, root_path):
    to_pil = transforms.ToPILImage()
    for idx, (image, label) in enumerate(dataset):
        class_path = os.path.join(root_path, str(label))
        os.makedirs(class_path, exist_ok=True)
        image_path = os.path.join(class_path, f'{idx}.png')
        pil_image = to_pil(image)
        pil_image.save(image_path)

In [7]:
save_images(trainset, train_path)
save_images(valset, val_path)

#1. Models

In [8]:
!git clone https://github.com/gosh-a/solo-learn.git
%cd solo-learn
!pip install .

Cloning into 'solo-learn'...
remote: Enumerating objects: 5164, done.
remote: Counting objects: 100% (297/297), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 5164 (delta 177), reused 191 (delta 116), pack-reused 4867
Receiving objects: 100% (5164/5164), 5.21 MiB | 16.61 MiB/s, done.
Resolving deltas: 100% (3636/3636), done.
/content/solo-learn
Processing /content/solo-learn
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 52.9 MB/s eta

In [ ]:
!nvidia-smi

Sat Jun 22 22:03:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Simclr

### pretrain the model

In [2]:
command = ['python3', 'solo-learn/main_pretrain.py',
           '--config-path', 'scripts/pretrain/cifar/',
           '--config-name', 'simclr.yaml',
           '++wandb.offline=True']

In [3]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

for line in process.stdout:
    print(line, end='')

for line in process.stderr:
    print(line, end='')

Files already downloaded and verified

Training: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 195/195 [01:50<00:00,  1.77it/s, v_num=572k]
/content/solo-learn/solo/backbones/convnext/convnext.py:27: UserWarning: Overwriting convnext_tiny in registry with solo.backbones.convnext.convnext.convnext_tiny. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def convnext_tiny(**kwargs):
/content/solo-learn/solo/backbones/convnext/convnext.py:34: UserWarning: Overwriting convnext_small in registry with solo.backbones.convnext.convnext.convnext_small. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def convnext_small(**kwargs):
/content/solo-learn/solo/backbones/convnext/convnext.py:41: UserWarning: Overwriting convnext_base in registry with solo.backbones.convnext.convnext.convnext_base. This is because the name being registered conflicts with an exi

### evaluate the model

#### linear evaluation

In [4]:
for dirpath, _, filenames in os.walk(os.path.join(os.getcwd(), 'trained_models', 'simclr')):
  if filenames:
    assert filenames[1].endswith('ckpt')
    new_path = os.path.join(dirpath, filenames[1].replace('=', '_'))
    os.rename(os.path.join(dirpath, filenames[1]), new_path)

In [5]:
command_path_name = '/'.join(new_path.split('/')[2:])

In [6]:
command = ['python3', 'solo-learn/main_linear.py',
           '--config-path', 'scripts/linear/cifar/',
           '--config-name', 'simclr.yaml',
           f'++pretrained_feature_extractor={command_path_name}',
           '++wandb.offline=True']

In [8]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

for line in process.stdout:
    print(line, end='')

for line in process.stderr:
    print(line, end='')

Files already downloaded and verified
Files already downloaded and verified

Sanity Checking: |          | 0/? [00:00<?, ?it/s]
Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  3.40it/s]
                                                                           

Training: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 195/195 [00:26<00:00,  7.23it/s, v_num=pkao]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:  50%|█████     | 20/40 [00:01<00:01, 12.02it/s]

Validation DataLoader 0: 100%|██████████| 40/40 [00:02<00:00, 13.52it/s]

                                                                        
Epoch 1: 100%|██████████| 195/195 [00:25<00:00,  7.69it/s, v_num=pkao]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:   0%|          | 0/40 

#### KNN evaluation

In [18]:
model_dir = '/'.join(command_path_name.split('/')[:-1])

In [19]:
command = ['python3', 'solo-learn/main_knn.py',
           '--dataset', 'cifar10',
           '--train_data_path', 'datasets/cifar10/train',
           '--val_data_path', 'datasets/cifar10/val',
           '--batch_size', '16',
           '--num_workers', '10',
           '--pretrained_checkpoint_dir', model_dir,
           '--k', '1', '2', '5', '10', '20', '50', '100', '200',
           '--temperature', '0.01', '0.02', '0.05', '0.07', '0.1', '0.2', '0.5', '1',
           '--feature_type', 'backbone', 'projector',
           '--distance_function', 'euclidean', 'cosine']

In [20]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

for line in process.stdout:
    print(line, end='')

for line in process.stderr:
    print(line, end='')

Files already downloaded and verified
Files already downloaded and verified

### BACKBONE ###
---
Running k-NN with params: distance_fx=euclidean, k=1, T=None...
Result: acc@1=48.55, acc@5=48.55
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.01...
Result: acc@1=4.46, acc@5=4.46
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.02...
Result: acc@1=48.53, acc@5=48.53
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.05...
Result: acc@1=48.53, acc@5=48.53
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.07...
Result: acc@1=48.53, acc@5=48.53
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.1...
Result: acc@1=48.53, acc@5=48.53
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.2...
Result: acc@1=48.53, acc@5=48.53
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.5...
Result: acc@1=48.53, acc@5=48.53
---
Running k-NN with params: distance_fx=cosine, k=1, T=1.0...
Result: acc@1=48.53, acc@5=48.53
---
Running k-NN with param

## MoCoV3

### pretrain the model

In [21]:
command = ['python3', 'solo-learn/main_pretrain.py',
           '--config-path', 'scripts/pretrain/cifar/',
           '--config-name', 'mocov3.yaml',
           '++wandb.offline=True']

In [22]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

for line in process.stdout:
    print(line, end='')

for line in process.stderr:
    print(line, end='')

Files already downloaded and verified

Training: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 195/195 [01:51<00:00,  1.75it/s, v_num=lyc7]
/content/solo-learn/solo/backbones/convnext/convnext.py:27: UserWarning: Overwriting convnext_tiny in registry with solo.backbones.convnext.convnext.convnext_tiny. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def convnext_tiny(**kwargs):
/content/solo-learn/solo/backbones/convnext/convnext.py:34: UserWarning: Overwriting convnext_small in registry with solo.backbones.convnext.convnext.convnext_small. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def convnext_small(**kwargs):
/content/solo-learn/solo/backbones/convnext/convnext.py:41: UserWarning: Overwriting convnext_base in registry with solo.backbones.convnext.convnext.convnext_base. This is because the name being registered conflicts with an exi

### evaluate the model

#### linear evaluation

In [23]:
for dirpath, _, filenames in os.walk(os.path.join(os.getcwd(), 'trained_models', 'mocov3')):
  if filenames:
    assert filenames[1].endswith('ckpt')
    new_path = os.path.join(dirpath, filenames[1].replace('=', '_'))
    os.rename(os.path.join(dirpath, filenames[1]), new_path)

In [24]:
command_path_name = '/'.join(new_path.split('/')[2:])

In [25]:
command = ['python3', 'solo-learn/main_linear.py',
           '--config-path', 'scripts/linear/cifar/',
           '--config-name', 'mocov3.yaml',
           f'++pretrained_feature_extractor={command_path_name}',
           '++wandb.offline=True']

In [26]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

for line in process.stdout:
    print(line, end='')

for line in process.stderr:
    print(line, end='')

Files already downloaded and verified
Files already downloaded and verified

Sanity Checking: |          | 0/? [00:00<?, ?it/s]
Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  3.21it/s]
                                                                           

Training: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 195/195 [00:24<00:00,  7.87it/s, v_num=exob]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:  50%|█████     | 20/40 [00:01<00:01, 19.66it/s]

Validation DataLoader 0: 100%|██████████| 40/40 [00:02<00:00, 18.59it/s]

                                                                        
Epoch 1: 100%|██████████| 195/195 [00:27<00:00,  7.15it/s, v_num=exob]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:   0%|          | 0/40 

#### KNN evaluation

In [27]:
model_dir = '/'.join(command_path_name.split('/')[:-1])

In [28]:
command = ['python3', 'solo-learn/main_knn.py',
           '--dataset', 'cifar10',
           '--train_data_path', 'datasets/cifar10/train',
           '--val_data_path', 'datasets/cifar10/val',
           '--batch_size', '16',
           '--num_workers', '10',
           '--pretrained_checkpoint_dir', model_dir,
           '--k', '1', '2', '5', '10', '20', '50', '100', '200',
           '--temperature', '0.01', '0.02', '0.05', '0.07', '0.1', '0.2', '0.5', '1',
           '--feature_type', 'backbone', 'projector',
           '--distance_function', 'euclidean', 'cosine']

In [29]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

for line in process.stdout:
    print(line, end='')

for line in process.stderr:
    print(line, end='')

Files already downloaded and verified
Files already downloaded and verified

### BACKBONE ###
---
Running k-NN with params: distance_fx=euclidean, k=1, T=None...
Result: acc@1=48.96, acc@5=48.96
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.01...
Result: acc@1=5.13, acc@5=5.13
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.02...
Result: acc@1=49.69, acc@5=49.69
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.05...
Result: acc@1=49.69, acc@5=49.69
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.07...
Result: acc@1=49.69, acc@5=49.69
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.1...
Result: acc@1=49.69, acc@5=49.69
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.2...
Result: acc@1=49.69, acc@5=49.69
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.5...
Result: acc@1=49.69, acc@5=49.69
---
Running k-NN with params: distance_fx=cosine, k=1, T=1.0...
Result: acc@1=49.69, acc@5=49.69
---
Running k-NN with param

## DINO

### pretrain the model

In [30]:
command = ['python3', 'solo-learn/main_pretrain.py',
           '--config-path', 'scripts/pretrain/cifar/',
           '--config-name', 'dino.yaml',
           '++wandb.offline=True']

In [31]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

for line in process.stdout:
    print(line, end='')

for line in process.stderr:
    print(line, end='')

Files already downloaded and verified

Training: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 195/195 [01:55<00:00,  1.69it/s, v_num=ckb6]
/content/solo-learn/solo/backbones/convnext/convnext.py:27: UserWarning: Overwriting convnext_tiny in registry with solo.backbones.convnext.convnext.convnext_tiny. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def convnext_tiny(**kwargs):
/content/solo-learn/solo/backbones/convnext/convnext.py:34: UserWarning: Overwriting convnext_small in registry with solo.backbones.convnext.convnext.convnext_small. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def convnext_small(**kwargs):
/content/solo-learn/solo/backbones/convnext/convnext.py:41: UserWarning: Overwriting convnext_base in registry with solo.backbones.convnext.convnext.convnext_base. This is because the name being registered conflicts with an exi

### evaluate the model

#### linear evalutaion

In [32]:
for dirpath, _, filenames in os.walk(os.path.join(os.getcwd(), 'trained_models', 'dino')):
  if filenames:
    assert filenames[1].endswith('ckpt')
    new_path = os.path.join(dirpath, filenames[1].replace('=', '_'))
    os.rename(os.path.join(dirpath, filenames[1]), new_path)

In [33]:
command_path_name = '/'.join(new_path.split('/')[2:])

In [34]:
command = ['python3', 'solo-learn/main_linear.py',
           '--config-path', 'scripts/linear/cifar/',
           '--config-name', 'dino.yaml',
           f'++pretrained_feature_extractor={command_path_name}',
           '++wandb.offline=True']

In [35]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

for line in process.stdout:
    print(line, end='')

for line in process.stderr:
    print(line, end='')

Files already downloaded and verified
Files already downloaded and verified

Sanity Checking: |          | 0/? [00:00<?, ?it/s]
Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  3.25it/s]
                                                                           

Training: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 195/195 [00:25<00:00,  7.77it/s, v_num=ka7j]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:  50%|█████     | 20/40 [00:00<00:00, 20.33it/s]

Validation DataLoader 0: 100%|██████████| 40/40 [00:02<00:00, 19.24it/s]

                                                                        
Epoch 1: 100%|██████████| 195/195 [00:29<00:00,  6.59it/s, v_num=ka7j]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:   0%|          | 0/40 

#### KNN evaluation

In [36]:
model_dir = '/'.join(command_path_name.split('/')[:-1])

In [37]:
command = ['python3', 'solo-learn/main_knn.py',
           '--dataset', 'cifar10',
           '--train_data_path', 'datasets/cifar10/train',
           '--val_data_path', 'datasets/cifar10/val',
           '--batch_size', '16',
           '--num_workers', '10',
           '--pretrained_checkpoint_dir', model_dir,
           '--k', '1', '2', '5', '10', '20', '50', '100', '200',
           '--temperature', '0.01', '0.02', '0.05', '0.07', '0.1', '0.2', '0.5', '1',
           '--feature_type', 'backbone', 'projector',
           '--distance_function', 'euclidean', 'cosine']

In [38]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

for line in process.stdout:
    print(line, end='')

for line in process.stderr:
    print(line, end='')

Files already downloaded and verified
Files already downloaded and verified

### BACKBONE ###
---
Running k-NN with params: distance_fx=euclidean, k=1, T=None...
Result: acc@1=33.81, acc@5=33.81
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.01...
Result: acc@1=7.57, acc@5=7.57
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.02...
Result: acc@1=34.56, acc@5=34.56
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.05...
Result: acc@1=34.56, acc@5=34.56
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.07...
Result: acc@1=34.56, acc@5=34.56
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.1...
Result: acc@1=34.56, acc@5=34.56
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.2...
Result: acc@1=34.56, acc@5=34.56
---
Running k-NN with params: distance_fx=cosine, k=1, T=0.5...
Result: acc@1=34.56, acc@5=34.56
---
Running k-NN with params: distance_fx=cosine, k=1, T=1.0...
Result: acc@1=34.56, acc@5=34.56
---
Running k-NN with param